| Übungszettel 7 | Einführung in die Numerik |
|---|---|
| Dozent      | Prof. Kanschat |
| Tutoren     | S. Meggendorfer und J. Witte |
| Abgabe      | bis 03.06.21 23.15 Uhr |
| Studierende | *bitte Vor- und Nachnamen einfügen* |

---

### Aufgabe 1: Ebene Kurve

Für $\alpha = 1$ und $\kappa = 2$ lässt sich eine [blumenförmige, ebene Kurve](https://de.wikipedia.org/wiki/Rosette_(Kurve)) darstellen durch

$$
X(t) = \alpha \cos(\kappa t) \cos(t), \\
Y(t) = \alpha \cos(\kappa t) \sin(t). \\
$$

Die Kurve ist periodisch bezüglich $2\pi$.

1. Berechnen Sie für (äquidistante) Stützstellen der Parametrisierung $t_0 = 0 < t_1 < \ldots < t_n = 2\pi$ mit $n > 7$ sowohl eine kubische Spline-Interpolation für $X(t)$ als auch für $Y(t)$. Dazu passen Sie lediglich die Funktion `solve_moments(...)` vom Übungsblatt 6 an, indem Sie die natürlichen Randbedingungen dort,
$$
2 M_0 = 0 , \\
2 M_n = 0 ,
$$
durch periodische Randbedingungen hier,
$$
M_0 = M_n, \\
\frac{h_n}{h_n + h_1} M_{n-1} + 2 M_n + \frac{h_1}{h_n + h_1} M_1
=
\frac{6}{h_n + h_1} \left[ \frac{f_1-f_n}{h_1} - \frac{f_n-f_{n-1}}{h_n} \right] ,
$$
für das lineare Gleichungssystem aus *Lemma 3.2.14* ersetzen.

2. Variieren Sie $n > 7$ und plotten Sie jeweils die Kurve $(X(t),Y(t))$ sowie die erhaltene Spline-Interpolierende.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

n = 8
a = 0.
b = 2.*np.pi

ALPHA = 1.
KAPPA = 2

---

In [ ]:
# bitte ergänzen